In [1]:
dir='/home/lisha/school/Projects/hyperband_nnet/hyperband2/mrbi/'
hyperband='hyperband/'
random='random/'
smac='smac/'
hyperopt='hyperopt/'
spearmint='spearmint/'


In [2]:
import pickle
import glob, os
import fnmatch


In [3]:
searchers={'hyperband':hyperband,'random':random, 'smac':smac,'hyperopt': hyperopt,'spearmint':spearmint}
filenames={'hyperband':'hyperband','random':'random_hyperopt_august2013_mod', 'smac':'smac_2_06_01-dev','hyperopt': 'hyperopt_august2013_mod','spearmint':'spearmint_april2013_mod'}
results={}
for s in searchers.keys():
    subdir=dir+searchers[s]
    os.chdir(subdir)
    results[s]={}
    for dirpath, dirnames, files in os.walk(subdir):
        for f in fnmatch.filter(files, s+'*.pkl'):
            if s =='hyperband':
                index_trial=dirpath.index('trial')+5
                seed=int(dirpath[index_trial:])
            else:
                index_filename= dirpath.index(filenames[s])+len(filenames[s])
                seed=int(dirpath[index_filename+1:dirpath.find('_',index_filename+1)])
            #(s=='hyperband' and seed < 6) for random_split
            results[s][seed]=dirpath+'/'+f
    
    

In [4]:
results

{'hyperband': {2300: '/home/lisha/school/Projects/hyperband_nnet/hyperband2/mrbi/hyperband/trial2300/hyperband_results.pkl',
  2500: '/home/lisha/school/Projects/hyperband_nnet/hyperband2/mrbi/hyperband/trial2500/hyperband_results.pkl',
  2600: '/home/lisha/school/Projects/hyperband_nnet/hyperband2/mrbi/hyperband/trial2600/hyperband_results.pkl',
  2700: '/home/lisha/school/Projects/hyperband_nnet/hyperband2/mrbi/hyperband/trial2700/hyperband_results.pkl',
  2800: '/home/lisha/school/Projects/hyperband_nnet/hyperband2/mrbi/hyperband/trial2800/hyperband_results.pkl'},
 'hyperopt': {1700: '/home/lisha/school/Projects/hyperband_nnet/hyperband2/mrbi/hyperopt/hyperopt_august2013_mod_1700_2016-5-6--21-50-17-314556/hyperopt_august2013_mod.pkl',
  2700: '/home/lisha/school/Projects/hyperband_nnet/hyperband2/mrbi/hyperopt/hyperopt_august2013_mod_2700_2016-5-6--22-1-58-120926/hyperopt_august2013_mod.pkl',
  3700: '/home/lisha/school/Projects/hyperband_nnet/hyperband2/mrbi/hyperopt/hyperopt_augus

In [5]:
test_error_append={}
for s in ['smac','hyperopt','spearmint','random']:
    subdir=dir+searchers[s]
    os.chdir(subdir)
    test_error_append[s]={}
    for dirpath, dirnames, files in os.walk(subdir):
        for f in fnmatch.filter(files, 'hyperband_run.log'):
            index_filename= dirpath.index(filenames[s])+len(filenames[s])
            seed=int(dirpath[index_filename+1:dirpath.find('_',index_filename+1)])
            test_error_append[s][seed]=[]
            with open(dirpath+"/"+f,'r') as file:
                for x in file:
                    search_str='Result for ParamILS: SAT'
                    if x[:len(search_str)]==search_str:
                        test_err=x[x.find('test_error,')+len('test_error, '):]
                        test_error_append[s][seed].append(float(test_err))
                        

In [6]:
seeds={'hyperband':3,'random':3, 'smac':100,'hyperopt': 100,'spearmint':100}
data={}
for s in results.keys():
    data[s]={}
    for seed in results[s].keys():
        data[s][seed] = pickle.load(open(results[s][seed],'r'))
    

In [7]:
def get_hyperband_results(dictionary):
    data=dictionary[0]
    hyperband_results=[]
    val_err=1
    test_err=1
    times=[4, 8, 12, 16, 24,32,40,48,56]
    #times=[4, 8, 12, 16, 20,28,36,44,52,60]
    indices=[0,1,2,3,5,6,7,8,9]
    for i in range(len(times)):
        time=times[i]
        index=indices[i]
        if 1- data[index][1][0]['results'][-1][2] < val_err:
            val_err = 1- data[index][1][0]['results'][-1][2]
            test_err = 1-data[index][1][0]['results'][-1][3]
        hyperband_results.append([time,val_err,test_err])
    return hyperband_results

In [8]:
def get_random_results(data,speed=1):
    #data=dictionary[0]
    random_results=[]
    n_arms=len(data)
    val_err=1
    test_err=1
    
    for i in range(n_arms):
        time = (i+1)*1.0/speed
        if data[i]['result']<val_err:
            val_err = data[i]['result']
            test_err = data[i]['test_error']
        random_results.append([time,val_err,test_err])
    return random_results

In [9]:
def minutes(time,start):
    return (time-start)/60.
def get_results_hpo(data):
    output=[]
    val_err=1
    test_err=1
    for i in range(len(data['cv_endtime'])):
        dur = i+1
        if data['trials'][i]['result'] < val_err:
            val_err=data['trials'][i]['result']
            test_err=data['trials'][i]['test_error']
        output.append([dur,val_err,test_err])
    return output
min_results={}
for s in ['random','hyperband','smac','hyperopt','spearmint']:
    min_results[s]={}
    for seed in data[s].keys():
        #if s =='random':
        #    min_results[s][seed]=get_random_results(data[s][seed][0],1)
        if s =='hyperband':
            min_results[s][seed]=get_hyperband_results(data[s][seed])
        else:
            min_results[s][seed]=get_results_hpo(data[s][seed])
    
        
min_results['random_2x']={}
for seed in range(len(data['random'].keys())/2):
    seed1=data['random'].keys()[2*seed]
    seed2=data['random'].keys()[2*seed+1]
    random_data=data['random'][seed1]['trials']+data['random'][seed2]['trials']
    min_results['random_2x'][seed1]=get_random_results(random_data,2)

In [10]:
import numpy
def average_trials(s_data):
    mean_data={}
    for s in s_data.keys():
        mean_data[s]=[]
        min_iters=min([len(s_data[s][seed]) for seed in s_data[s].keys()])
        for i in range(min_iters):
            avg_r=numpy.nanmean([min_results[s][seed][i] for seed in min_results[s].keys()],axis=0)
            max_r=numpy.nanmax([min_results[s][seed][i] for seed in min_results[s].keys()],axis=0)
            max_r[1:]=max_r[1:]-avg_r[1:]
            min_r=numpy.nanmin([min_results[s][seed][i] for seed in min_results[s].keys()],axis=0)
            min_r[1:]=avg_r[1:]-min_r[1:]
            mean_data[s].append([min_r,avg_r,max_r])
    return mean_data

In [11]:
mean_data=average_trials(min_results)
mean_data['spearmint']

[[array([ 1.      ,  0.4824  ,  0.491216]),
  array([ 1.      ,  0.7624  ,  0.765976]),
  array([ 1.      ,  0.1206  ,  0.122804])],
 [array([ 2.      ,  0.2765  ,  0.279388]),
  array([ 2.      ,  0.5565  ,  0.554148]),
  array([ 2.      ,  0.3265  ,  0.334632])],
 [array([ 3.      ,  0.2765  ,  0.279388]),
  array([ 3.      ,  0.5565  ,  0.554148]),
  array([ 3.      ,  0.3265  ,  0.334632])],
 [array([ 4.      ,  0.2765  ,  0.279388]),
  array([ 4.      ,  0.5565  ,  0.554148]),
  array([ 4.      ,  0.3265  ,  0.334632])],
 [array([ 5.      ,  0.2765  ,  0.279388]),
  array([ 5.      ,  0.5565  ,  0.554148]),
  array([ 5.      ,  0.3265  ,  0.334632])],
 [array([ 6.      ,  0.2765  ,  0.279388]),
  array([ 6.      ,  0.5565  ,  0.554148]),
  array([ 6.      ,  0.3265  ,  0.334632])],
 [array([ 7.     ,  0.1726 ,  0.17264]),
  array([ 7.    ,  0.4526,  0.4474]),
  array([ 7.     ,  0.4304 ,  0.44138])],
 [array([ 8.     ,  0.1594 ,  0.15752]),
  array([ 8.     ,  0.4394 ,  0.43228]),

In [12]:
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot as plt


In [42]:
colors={'hyperband':'y','smac':'r','hyperopt':'b','random':'g','spearmint':'c','random_2x':'g'}
seeds={'hyperband':3,'random':3, 'smac':100,'hyperopt': 100,'spearmint':100}
def results_plot(index):
    for s in ['smac','hyperopt','hyperband','random_2x','spearmint']:
        if index==2:
            axis_label='Test Error'
        else:
            axis_label='Val Error'
        for seed_ind in range(1):
            seed=seeds[s]
            if min_results[s][seed][0][0]>0:
                min_results[s][seed].insert(0,[1,1,1])
            if seed_ind==0:
                plt.plot([t[0] for t in min_results[s][seed]],[t[index] for t in min_results[s][seed]],color=colors[s])
            else:
                plt.plot([t[0] for t in min_results[s][seed]],[t[index] for t in min_results[s][seed]],color=colors[s],
                         label=s)
        plt.legend()
        plt.xlabel('# Arms trained to Completion')
        plt.ylabel(axis_label)
    #plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
pdf=PdfPages('../mrbi_error_trial2.pdf')
plt.figure(figsize=(11,8.5))
plt.subplot(121)
results_plot(1)
plt.subplot(122)
results_plot(2)
plt.suptitle('Validation and Test Error on Cifar10')
pdf.savefig()
plt.close()
pdf.close()
#plt.show()


KeyError: 100

In [18]:
colors={'hyperband':'y','smac':'r','hyperopt':'b','random':'g','spearmint':'c','random_2x':'g'}
seeds={'hyperband':3,'random':3, 'smac':100,'hyperopt': 100,'spearmint':100}
labels={'hyperband':'hyperband','smac':'smac','hyperopt':'TPE','random':'random','spearmint':'spearmint','random_2x':'random_2x'}
def results_plot(index):
    matplotlib.rcParams.update({'font.size': 16})
    line_width=2
    edge_width=2
    shift = 0.3
    for s in ['random_2x','smac','hyperopt','hyperband','spearmint']:
        if index==2:
            axis_label='Average Test Error'
        else:
            axis_label='Average Val Error'
        if s == 'hyperband':
            #plt.errorbar([i[0][0] for i in mean_data[s]],[i[1][index] for i in mean_data[s]],color=colors[s],
            #              yerr=[[i[0][index] for i in mean_data[s]],[i[2][index] for i in mean_data[s]]],
            #         label=s,linewidth=line_width,marker='x',markeredgewidth=edge_width,elinewidth=0.5,capthick=0.5,ecolor='w')
            plt.errorbar([i[0][0] for i in mean_data[s]],[i[1][index] for i in mean_data[s]],color=colors[s],                          
                     label=s,linewidth=line_width,marker='x',markeredgewidth=edge_width)
            #+' # trials = '+str(len(min_results[s].keys()))
            #x=[i[0][0] for i in mean_data[s]]
            #y=numpy.array([i[1][index] for i in mean_data[s]])
            #y_min=y-numpy.array([i[0][index] for i in mean_data[s]])
            #y_max=y+numpy.array([i[2][index] for i in mean_data[s]])
            #plt.plot(x,y,color=colors[s],
            #              label=s+' # trials = '+str(len(min_results[s].keys())))
            #plt.fill_between(x,y_min,y_max,alpha=0.5,edgecolor='y',facecolor='y')
        elif s=='random_2x':
            #plt.errorbar([row[0][0] for row in mean_data[s]],[row[1][index] for row in mean_data[s]],color=colors[s],errorevery=8,
            #              yerr=[[row[0][index] for row in mean_data[s]],
            #                    [row[2][index] for row in mean_data[s]]],label=s,linewidth=line_width,marker='+',markeredgewidth=edge_width,elinewidth=0.5,capthick=0.5,markevery=4,ecolor='w')
            plt.plot([row[0][0] for row in mean_data[s]],[row[1][index] for row in mean_data[s]],color=colors[s],
                          label=s,linewidth=line_width,marker='+',markeredgewidth=edge_width,markevery=4)
        else:
            #plt.plot([i[0][0] for i in mean_data[s]],[i[1][index] for i in mean_data[s]],color=colors[s],                          
            #         label=s+' # trials = '+str(len(min_results[s].keys())))
            #x=[row[0][0] for row in mean_data[s]]
            #y=numpy.array([row[1][index] for row in mean_data[s]])
            #y_min=y-numpy.array([row[0][index] for row in mean_data[s]])
            #y_max=y+numpy.array([row[2][index] for row in mean_data[s]])
            #plt.errorbar(numpy.array([row[0][0]+shift for row in mean_data[s]]),[row[1][index] for row in mean_data[s]],color=colors[s],errorevery=4,
            #              yerr=[[row[0][index] for row in mean_data[s]],
            #                    [row[2][index] for row in mean_data[s]]],label=s,linewidth=line_width,elinewidth=0.5,capthick=0.5,ecolor='w')
            plt.errorbar(numpy.array([row[0][0] for row in mean_data[s]]),[row[1][index] for row in mean_data[s]],color=colors[s],
                          label=labels[s],linewidth=line_width)
            shift+=0.3
    plt.legend(ncol=2,columnspacing=0.2,fancybox=True, framealpha=0.75)
    plt.xlim([3,57])
    plt.ylim([0.225,0.375])
    plt.xlabel('Multiple of Max Iter Used')
    plt.ylabel(axis_label)
    #plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
pdf=PdfPages('../MRBI_error_avg.pdf')
plt.figure(figsize=(6.5,5.5))
#plt.subplot(121)
#results_plot(1)
#plt.subplot(122)
results_plot(2)
#plt.suptitle('Test Error on CIFAR-10')
pdf.savefig()
plt.close()
pdf.close()
#plt.show()


In [299]:
shift

NameError: name 'shift' is not defined

In [171]:
[(max([i[2] for i in data['random'][s][0]]),max([i[3] for i in data['random'][s][0]])) for s in data['random'].keys()]

[(array(0.7599999904632568, dtype=float32),
  array(0.8199999928474426, dtype=float32)),
 (array(0.7099999785423279, dtype=float32), array(0.75, dtype=float32)),
 (array(0.7099999785423279, dtype=float32),
  array(0.8100000023841858, dtype=float32)),
 (array(0.699999988079071, dtype=float32),
  array(0.7799999713897705, dtype=float32)),
 (array(0.699999988079071, dtype=float32),
  array(0.7099999785423279, dtype=float32)),
 (array(0.7799999713897705, dtype=float32),
  array(0.8299999833106995, dtype=float32)),
 (array(0.6800000071525574, dtype=float32),
  array(0.7900000214576721, dtype=float32)),
 (array(0.699999988079071, dtype=float32),
  array(0.7900000214576721, dtype=float32)),
 (array(0.7900000214576721, dtype=float32),
  array(0.8299999833106995, dtype=float32))]